# CROSSCHECK ALL DATA

In [ ]:
pip install pandas plotly

In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import plotly.express as px

def get_centroid(file_path):
    """Mengambil koordinat rata-rata (centroid) dari file .osm"""
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        lats = []
        lons = []
        
        # Cara cepat: Cek tag <bounds> jika ada
        bounds = root.find('bounds')
        if bounds is not None:
            minlat = float(bounds.get('minlat'))
            maxlat = float(bounds.get('maxlat'))
            minlon = float(bounds.get('minlon'))
            maxlon = float(bounds.get('maxlon'))
            return (minlat + maxlat) / 2, (minlon + maxlon) / 2
        
        # Cara fallback: Ambil semua node dan rata-ratakan (lebih lambat)
        for node in root.findall('node'):
            lats.append(float(node.get('lat')))
            lons.append(float(node.get('lon')))
            
        if lats and lons:
            return sum(lats) / len(lats), sum(lons) / len(lons)
    except Exception as e:
        print(f"Error membaca {file_path}: {e}")
    return None, None

# 1. List nama file sesuai pola Anda
file_names = ["map.osm"] + [f"map ({i}).osm" for i in range(1, 38)]

data = []

# 2. Proses tiap file
print("Sedang memproses file...")
for name in file_names:
    if os.path.exists(name):
        lat, lon = get_centroid(name)
        if lat is not None:
            data.append({
                'File': name,
                'Latitude': lat,
                'Longitude': lon
            })
    else:
        print(f"File {name} tidak ditemukan.")

# 3. Buat DataFrame
df = pd.DataFrame(data)

# 4. Visualisasi dengan Mapbox
if not df.empty:
    fig = px.scatter_mapbox(
        df, 
        lat="Latitude", 
        lon="Longitude", 
        hover_name="File",
        zoom=10, 
        height=600,
        title="Persebaran Centroid Data OSM"
    )

    # Gunakan style open-street-map agar tidak perlu API Key Mapbox
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    
    fig.show()
    # Jika ingin simpan ke HTML: fig.write_html("peta_centroid.html")
else:
    print("Tidak ada data untuk ditampilkan.")

# PAKAI CARA BARU (KEKNYA SALAH)

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import os

def parse_osm_to_list(file_path):
    """Fungsi untuk memparsing file .osm menjadi list of dictionaries."""
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        data_list = []

        # Iterasi melalui elemen node, way, dan relation
        for elem in root:
            if elem.tag in ['node', 'way', 'relation']:
                # Ambil atribut dasar
                item = {
                    'element_type': elem.tag,
                    'id': elem.get('id'),
                    'lat': elem.get('lat'),
                    'lon': elem.get('lon'),
                    'user': elem.get('user'),
                    'timestamp': elem.get('timestamp')
                }
                
                # Ambil semua tag (kunci 'k' dan nilai 'v') di dalam elemen tersebut
                for tag in elem.findall('tag'):
                    key = tag.get('k')
                    value = tag.get('v')
                    item[key] = value
                
                data_list.append(item)
        return data_list
    except Exception as e:
        print(f"Error saat memproses {file_path}: {e}")
        return []

# List untuk menampung semua data dari 38 file
combined_data = []

print("Memulai proses konversi...")

for i in range(38):
    # Logika penamaan file: map.osm, map (1).osm, ..., map (37).osm
    if i == 0:
        filename = "map.osm"
    else:
        filename = f"map ({i}).osm"
    
    if os.path.exists(filename):
        print(f"Memproses: {filename}")
        file_data = parse_osm_to_list(filename)
        combined_data.extend(file_data)
    else:
        print(f"Peringatan: {filename} tidak ditemukan.")

# Mengubah list menjadi DataFrame pandas
if combined_data:
    df = pd.DataFrame(combined_data)
    
    # Simpan ke satu file CSV
    output_name = "exposure.csv"
    df.to_csv(output_name, index=False)
    print(f"\nBerhasil! Semua data telah digabung ke dalam: {output_name}")
    print(f"Total baris data: {len(df)}")
else:
    print("\nTidak ada data yang berhasil diproses.")

In [ ]:
import pandas as pd

# Membaca file CSV
df = pd.read_csv('exposure.csv')

# Menampilkan 5 baris pertama
print(df.head())

# Jika ingin melihat lebih banyak, misal 10 baris:
# print(df.head(10))

# PAKAI CARA AWAL

In [1]:
import pandas as pd
import geopandas as gpd
import os
import glob

# 1. Mencari semua file map.osm sampai map (37).osm
# Kita gunakan pattern untuk menangkap file dengan nama 'map.osm' dan 'map (*).osm'
file_list = glob.glob("map*.osm")

# 2. Menggabungkan data (daripada menggabungkan file mentah, lebih aman menggabungkan GeoDataFrame)
all_buildings = []

for file in sorted(file_list):
    try:
        # Membaca layer multipolygons (bangunan) per file
        # Catatan: Memerlukan driver OSM di geopandas/fiona
        gdf = gpd.read_file(file, layer='multipolygons')
        all_buildings.append(gdf)
        print(f"Berhasil membaca {file}")
    except Exception as e:
        print(f"Gagal membaca {file}: {e}")

# 3. Gabungkan semua menjadi satu GeoDataFrame
if all_buildings:
    combined_gdf = pd.concat(all_buildings, ignore_index=True)
    
    # 4. Simpan ke CSV sesuai request
    combined_gdf.to_csv("osm_exposure_plus_kuburaya.csv", index=False)
    print("Berhasil menyatukan data ke osm_exposure_plus_kuburaya.csv")
else:
    print("Tidak ada data yang berhasil dibaca.")

Berhasil membaca map (1).osm
Berhasil membaca map (10).osm
Berhasil membaca map (11).osm
Berhasil membaca map (12).osm
Berhasil membaca map (13).osm
Berhasil membaca map (14).osm
Berhasil membaca map (15).osm
Berhasil membaca map (16).osm
Berhasil membaca map (17).osm
Berhasil membaca map (18).osm
Berhasil membaca map (19).osm
Berhasil membaca map (2).osm
Berhasil membaca map (20).osm
Berhasil membaca map (21).osm
Berhasil membaca map (22).osm
Berhasil membaca map (23).osm
Berhasil membaca map (24).osm
Berhasil membaca map (25).osm
Berhasil membaca map (26).osm
Berhasil membaca map (27).osm
Berhasil membaca map (28).osm
Berhasil membaca map (29).osm
Berhasil membaca map (3).osm
Berhasil membaca map (30).osm
Berhasil membaca map (31).osm
Berhasil membaca map (32).osm
Berhasil membaca map (33).osm
Berhasil membaca map (34).osm
Berhasil membaca map (35).osm
Berhasil membaca map (36).osm
Berhasil membaca map (37).osm
Berhasil membaca map (38).osm
Berhasil membaca map (39).osm
Berhasil memb